In [ ]:
import pandas as pd
from binance.client import Client
from binance.exceptions import BinanceAPIException
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_SECRET_KEY")

In [ ]:
client = Client(api_key, api_secret)
client

In [ ]:
account = client.get_account()
account

In [ ]:
df = pd.DataFrame(account["balances"])
df

In [ ]:
df.free = pd.to_numeric(df.free, errors="coerce")

In [ ]:
current_assets = df.loc[df.free > 0]
current_assets

In [ ]:
# Fetch balances with more than zero balance
balances = {asset['asset']: float(asset['free']) + float(asset['locked'])
            for asset in client.get_account()['balances'] if float(asset['free']) + float(asset['locked']) > 0}

balances

In [ ]:
# Get current prices for all symbols
prices = {price['symbol']: float(price['price']) for price in client.get_all_tickers()}
prices

In [ ]:

# Estimate the USDT value of each asset
estimated_values = {}
for asset, balance in balances.items():
    if asset == 'USDT':
        estimated_values[asset] = balance
    elif f"{asset}USDT" in prices:
        estimated_values[asset] = balance * prices[f"{asset}USDT"]
    elif f"USDT{asset}" in prices:  # Handle reverse pairs
        estimated_values[asset] = balance / prices[f"USDT{asset}"]

estimated_values

In [ ]:

# Determine the asset with the highest estimated USDT value
most_valuable_symbol = max(estimated_values, key=estimated_values.get) if estimated_values else "No Transaction"
most_valuable_symbol